### 模型测试

In [1]:
import os
import joblib
# 加载模型和向量化器
model = joblib.load('rf_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

In [2]:
# 遍历文件夹中的邮件
emails, predictions = [], []
for root, dirs, files in os.walk('trec07p/data'):
    for file in files:
        # if file.endswith('.eml'):
        with open(os.path.join(root, file), 'r', encoding='utf-8', errors='ignore') as f:
            email_content = f.read()
        emails.append(email_content)

In [3]:
# 特征提取
email_features = vectorizer.transform(emails)
# 模型预测
predictions = model.predict(email_features)

In [5]:
# 计算垃圾邮件和非垃圾邮件的数量
spam_count = sum(predictions)
ham_count = len(predictions) - spam_count

# 计算占比
spam_ratio = spam_count / len(predictions)
ham_ratio = ham_count / len(predictions)

print(f'垃圾邮件占比: {spam_ratio:.2%}')
print(f'非垃圾邮件占比: {ham_ratio:.2%}')

垃圾邮件占比: 71.10%
非垃圾邮件占比: 28.90%
